In [ ]:
---
title: Comparison of Numerical Methods for ODE's
description: (1) Explicit Euler, (2) Explicit Euler, (3) RK2, (4) RK4
author: Daning H.
show-code: False
show-prompt: False
params:
    EE:
        label: Explicit Euler
        input: checkbox
        value: True
    IE:
        label: Implicit Euler
        input: checkbox
        value: True
    R2:
        label: Runge Kutta 2nd-order
        input: checkbox
        value: True
    R4:
        label: Runge Kutta 4th-order
        input: checkbox
        value: True
---

Here we solve the following ODE numerically
$$
y' = -20y+20x^2+2x, y(0)=1.0
$$
using four different methods: (1) Explicit Euler, (2) Explicit Euler, (3) RK2, (4) RK4.

Test and Observe:
1. When one increases the step size:
   + Which method produces spurious oscillations, and which do not?
   + Which method(s) diverge (i.e., grow without bound), and which do not?
2. When one decreases the step size:
   + How fast do the methods converge to the true solution?

In [8]:
EE = True
IE = True
R2 = True
R4 = False

In [9]:
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import root_scalar

T  = 0.5
y0 = 1.0
func = lambda y, t: -20*y+20*t*t+2*t
fsol = lambda t: np.exp(-20*t)+t*t

def update_plot(step):
    N  = int(T/step)+2
    ts = np.arange(N)*step
    y_ee = np.zeros((N,))
    y_ee[0] = y0
    y_ie = np.zeros((N,))
    y_ie[0] = y0
    y_rk = np.zeros((N,))
    y_rk[0] = y0
    y_r4 = np.zeros((N,))
    y_r4[0] = y0

    for _i in range(1,N):
        _t = ts[_i-1]

        # Exp. Euler
        y_ee[_i] = y_ee[_i-1] + step * func(y_ee[_i-1], _t)

        # Imp. Euler
        _f = lambda y: y_ie[_i-1] + step * func(y, ts[_i]) - y
        _sol = root_scalar(_f, x0=y_ie[_i-1], x1=2.0)
        y_ie[_i] = _sol.root

        # RK2
        _k1 = step * func(y_rk[_i-1], _t)
        _k2 = step * func(y_rk[_i-1]+0.5*_k1, _t+0.5*step)
        y_rk[_i] = y_rk[_i-1] + _k2

        # RK4
        _k1 = step * func(y_r4[_i-1], _t)
        _k2 = step * func(y_r4[_i-1]+0.5*_k1, _t+0.5*step)
        _k3 = step * func(y_r4[_i-1]+0.5*_k2, _t+0.5*step)
        _k4 = step * func(y_r4[_i-1]+_k3, _t+step)
        y_r4[_i] = y_r4[_i-1] + (_k1+2*_k2+2*_k3+_k4) / 6

    return ts, y_ee, y_ie, y_rk, y_r4

In [7]:
fig = go.Figure()

stps = np.arange(0.01, 0.16, 0.01)
Nstp = len(stps)
Nmet = 4
actv = 3
msk  = np.arange(4)[[EE, IE, R2, R4]]

# Add traces, one for each slider step
for step in stps:
    _t, _ye, _yi, _yr, _y4 = update_plot(step)
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="blue", width=2),
            name="Explicit Euler", x=_t, y=_ye))
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="red", width=2),
            name="Implicit Euler", x=_t, y=_yi))
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="green", width=2),
            name="RK2", x=_t, y=_yr))
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="grey", width=2),
            name="RK4", x=_t, y=_y4))
ts = np.linspace(0, T, 100)
fig.add_trace(
    go.Scatter(
        visible=True, line=dict(color="black", width=3, dash='dash'),
        name="Exact sol'n", x=ts, y=fsol(ts)))

for i in msk:
    fig.data[actv*Nmet+i].visible = True

# Create and add slider
steps = []
for i in range(Nstp):
    step = dict(
        method="update",
        args=[{"visible": [False] * (Nstp*Nmet) + [True]}],
        label=f"{stps[i]:3.2f}"
    )
    for j in msk:
        step["args"][0]["visible"][i*Nmet+j] = True
    steps.append(step)

sliders = [dict(
    active=actv,
    currentvalue={"prefix": "Step size = "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    xaxis_title="x",
    yaxis_title="y",
    #title=r"ODE: $y'=-20y+20x^2+2x, y(0)=1.0$"
)
fig.update_xaxes(range=[-0.02, 0.52])

fig.show()